In [2]:
!pip install facenet-pytorch

In [3]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import pandas as pd
import os

workers = 0 if os.name == 'nt' else 4

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [5]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

In [5]:
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

In [6]:
def collate_fn(x):
    return x[0]

dataset = datasets.ImageFolder('Images')
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers)

In [7]:
aligned = []
names = []
for x, y in loader:
    x_aligned, prob = mtcnn(x, return_prob=True)
    if x_aligned is not None:
        print('Face detected with probability: {:8f}'.format(prob))
        aligned.append(x_aligned)
        names.append(dataset.idx_to_class[y])

Face detected with probability: 0.999996
Face detected with probability: 0.999976
Face detected with probability: 0.999995
Face detected with probability: 0.999998
Face detected with probability: 0.999997
Face detected with probability: 0.999997
Face detected with probability: 1.000000
Face detected with probability: 0.999999
Face detected with probability: 0.999996
Face detected with probability: 0.999999
Face detected with probability: 1.000000
Face detected with probability: 1.000000
Face detected with probability: 0.999946
Face detected with probability: 0.999937
Face detected with probability: 0.999775
Face detected with probability: 0.999976
Face detected with probability: 0.999907
Face detected with probability: 0.999952
Face detected with probability: 0.999534
Face detected with probability: 0.999474
Face detected with probability: 0.999831
Face detected with probability: 0.999953
Face detected with probability: 0.999956
Face detected with probability: 0.999983
Face detected wi

In [8]:
aligned = torch.stack(aligned).to(device)
embeddings = resnet(aligned).detach().cpu()

In [9]:
dists = [[(e1 - e2).norm().item() for e2 in embeddings] for e1 in embeddings]
print(pd.DataFrame(dists, columns=names, index=names))

          101       101       101       103       103       103       106  \
101  0.000000  0.251937  0.245566  1.386807  1.391234  1.426390  1.234201   
101  0.251937  0.000000  0.176368  1.448362  1.449414  1.469912  1.205579   
101  0.245566  0.176368  0.000000  1.418605  1.427021  1.443167  1.215500   
103  1.386807  1.448362  1.418605  0.000000  0.240004  0.358194  1.504586   
103  1.391234  1.449414  1.427021  0.240004  0.000000  0.296569  1.523459   
..        ...       ...       ...       ...       ...       ...       ...   
96   1.304131  1.350018  1.305225  1.215658  1.291156  1.254711  1.410383   
96   1.416349  1.457923  1.401473  1.189474  1.257473  1.199917  1.498320   
97   1.441590  1.458378  1.427107  1.063741  1.127900  1.106664  1.453432   
97   1.462412  1.466486  1.438400  1.037010  1.093780  1.074750  1.464691   
97   1.451035  1.450113  1.423538  1.041426  1.099166  1.062352  1.487652   

          106       106       107  ...        92        95        95  \
101

In [10]:
faces = embeddings.numpy()

In [11]:
len(faces)
#75 faces in object

75

In [12]:
import json
from json import JSONEncoder
import numpy

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)



# Serialization

innit_id = 0
numpyData = []

for itr, ele in enumerate(faces):
    numpyData.append({"id":itr, "array": ele})

#numpyData = {"array": testing}
encodedNumpyData = json.dumps(numpyData, cls=NumpyArrayEncoder)  # use dump() to write array into file
print("Printing JSON serialized NumPy array")
#print(encodedNumpyData)

# Deserialization
print("Decode JSON serialized NumPy array")
decodedArrays = json.loads(encodedNumpyData)


#finalNumpyArray = itr, ele
#print (finalNumpyArray)

face_id = []
face_vector = []

for ele in decodedArrays:
    face_id.append(ele['id'])
    face_vector.append(ele['array'])


Printing JSON serialized NumPy array
Decode JSON serialized NumPy array


In [13]:
face_id

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74]

In [14]:
face_vector[3]

[0.02554946392774582,
 0.007887429557740688,
 0.0034096012823283672,
 -0.08392559736967087,
 -0.014644832350313663,
 -0.042760420590639114,
 0.025261370465159416,
 0.008560211397707462,
 -0.0024920525029301643,
 0.023834403604269028,
 0.08125220984220505,
 -0.07309892028570175,
 0.001439704792574048,
 -0.004887416493147612,
 0.02028827741742134,
 -0.058208443224430084,
 -0.08301622420549393,
 0.002741029253229499,
 -0.11728474497795105,
 -0.015576984733343124,
 0.05782974511384964,
 0.0020706651266664267,
 -0.021941617131233215,
 -0.03798600286245346,
 -0.0944729670882225,
 -0.03799491375684738,
 -0.041129060089588165,
 -0.05235164612531662,
 -0.04781649634242058,
 -0.07743226736783981,
 0.03409368917346001,
 0.04986635223031044,
 0.07037226855754852,
 0.019110819324851036,
 -0.014265656471252441,
 0.0546150803565979,
 0.0372496172785759,
 0.053039465099573135,
 0.021978093311190605,
 -0.0915602296590805,
 -0.03209248557686806,
 -0.020091086626052856,
 -0.018100786954164505,
 0.0114169

In [15]:
from scipy.spatial import distance

In [16]:
distance.euclidean(face_vector[0], face_vector[2])

0.2455656681133439

In [18]:
detection_threshold = 0.5